<a href="https://colab.research.google.com/github/albarker17/deep-learning-challenge/blob/main/AlphabetSoupCharity_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns = ["EIN", "NAME"])
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [4]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [5]:
# Drop the non-beneficial ID columns, 'ASK_AMT	'.
application_df = application_df.drop(columns = ['ASK_AMT'])
application_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,1
...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,1


In [6]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE          17
AFFILIATION                6
CLASSIFICATION            71
USE_CASE                   5
ORGANIZATION               4
STATUS                     2
INCOME_AMT                 9
SPECIAL_CONSIDERATIONS     2
IS_SUCCESSFUL              2
dtype: int64

In [7]:
# Look at APPLICATION_TYPE value counts for binning
application_type_counts = application_df["APPLICATION_TYPE"].value_counts()
application_type_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [8]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(application_type_counts[application_type_counts<750].index)
application_types_to_replace

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
Other     2266
T4        1542
T6        1216
T5        1173
T19       1065
Name: APPLICATION_TYPE, dtype: int64

In [9]:
# Look at CLASSIFICATION value counts for binning
classification_value_counts = application_df["CLASSIFICATION"].value_counts()
classification_value_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [10]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(classification_value_counts[classification_value_counts<200].index)
classifications_to_replace

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1197
C7000      777
C1700      287
Name: CLASSIFICATION, dtype: int64

In [11]:
dummies = pd.get_dummies(application_df)
dummies.head()

,STATUS,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,1,0,0,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,0,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,1,0,0,1,0,0,0,1,0,...,0,1,0,0,0,0,0,0,1,0
4,1,1,0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [12]:
# Split our preprocessed data into our features and target arrays
y = dummies["IS_SUCCESSFUL"].values
X = dummies.drop(["IS_SUCCESSFUL"],1).values



# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

<ipython-input-12-716494e7ce7a>:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X = dummies.drop(["IS_SUCCESSFUL"],1).values


In [13]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [14]:
#ATTEMPT 1

number_input_features = len(X_train[0])
hidden_nodes_layer1 = 90
hidden_nodes_layer2 = 45

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 90)                3780      
                                                                 
 dense_1 (Dense)             (None, 45)                4095      
                                                                 
 dense_2 (Dense)             (None, 1)                 46        
                                                                 
Total params: 7,921
Trainable params: 7,921
Non-trainable params: 0
_________________________________________________________________


In [15]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [16]:
# Train the model
fit_model=nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5713 - accuracy: 0.7218
Epoch 2/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5580 - accuracy: 0.7289
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5555 - accuracy: 0.7304
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5528 - accuracy: 0.7307
Epoch 5/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5514 - accuracy: 0.7319
Epoch 6/100
804/804 [==============================] - 4s 5ms/step - loss: 0.5505 - accuracy: 0.7322
Epoch 7/100
804/804 [==============================] - 5s 6ms/step - loss: 0.5505 - accuracy: 0.7330
Epoch 8/100
804/804 [==============================] - 5s 6ms/step - loss: 0.5496 - accuracy: 0.7338
Epoch 9/100
804/804 [==============================] - 6s 7ms/step - loss: 0.5490 - accuracy: 0.7348
Epoch 10/100
804/804 [==============================] - 5s 6ms/step - loss: 0.5482 - accura

In [17]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5597 - accuracy: 0.7261 - 804ms/epoch - 3ms/step
Loss: 0.5597238540649414, Accuracy: 0.726064145565033


In [18]:
#ATTEMPT 2

number_input_features = len(X_train[0])
hidden_nodes_layer1 = 15
hidden_nodes_layer2 = 10

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 15)                630       
                                                                 
 dense_4 (Dense)             (None, 10)                160       
                                                                 
 dense_5 (Dense)             (None, 1)                 11        
                                                                 
Total params: 801
Trainable params: 801
Non-trainable params: 0
_________________________________________________________________


In [19]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [20]:
# Train the model
fit_model=nn.fit(X_train_scaled, y_train, epochs=125)

Epoch 1/125
804/804 [==============================] - 4s 3ms/step - loss: 0.6267 - accuracy: 0.6734
Epoch 2/125
804/804 [==============================] - 2s 2ms/step - loss: 0.5682 - accuracy: 0.7270
Epoch 3/125
804/804 [==============================] - 2s 2ms/step - loss: 0.5614 - accuracy: 0.7313
Epoch 4/125
804/804 [==============================] - 2s 2ms/step - loss: 0.5583 - accuracy: 0.7318
Epoch 5/125
804/804 [==============================] - 2s 2ms/step - loss: 0.5559 - accuracy: 0.7327
Epoch 6/125
804/804 [==============================] - 2s 2ms/step - loss: 0.5538 - accuracy: 0.7324
Epoch 7/125
804/804 [==============================] - 3s 3ms/step - loss: 0.5519 - accuracy: 0.7335
Epoch 8/125
804/804 [==============================] - 2s 2ms/step - loss: 0.5514 - accuracy: 0.7327
Epoch 9/125
804/804 [==============================] - 2s 2ms/step - loss: 0.5507 - accuracy: 0.7332
Epoch 10/125
804/804 [==============================] - 2s 2ms/step - loss: 0.5504 - accura

In [21]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5556 - accuracy: 0.7262 - 1s/epoch - 5ms/step
Loss: 0.5556126236915588, Accuracy: 0.7261807322502136


In [23]:
#ATTEMPT 3

number_input_features = len(X_train[0])
hidden_nodes_layer1 = 40
hidden_nodes_layer2 = 40

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 40)                1680      
                                                                 
 dense_10 (Dense)            (None, 40)                1640      
                                                                 
 dense_11 (Dense)            (None, 1)                 41        
                                                                 
Total params: 3,361
Trainable params: 3,361
Non-trainable params: 0
_________________________________________________________________


In [24]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [25]:
# Train the model
fit_model=nn.fit(X_train_scaled, y_train, epochs=125)

Epoch 1/125
804/804 [==============================] - 4s 4ms/step - loss: 0.5795 - accuracy: 0.7147
Epoch 2/125
804/804 [==============================] - 2s 2ms/step - loss: 0.5579 - accuracy: 0.7304
Epoch 3/125
804/804 [==============================] - 2s 3ms/step - loss: 0.5548 - accuracy: 0.7309
Epoch 4/125
804/804 [==============================] - 2s 3ms/step - loss: 0.5529 - accuracy: 0.7312
Epoch 5/125
804/804 [==============================] - 2s 3ms/step - loss: 0.5517 - accuracy: 0.7319
Epoch 6/125
804/804 [==============================] - 2s 3ms/step - loss: 0.5504 - accuracy: 0.7336
Epoch 7/125
804/804 [==============================] - 3s 4ms/step - loss: 0.5499 - accuracy: 0.7324
Epoch 8/125
804/804 [==============================] - 2s 2ms/step - loss: 0.5489 - accuracy: 0.7347
Epoch 9/125
804/804 [==============================] - 2s 2ms/step - loss: 0.5488 - accuracy: 0.7325
Epoch 10/125
804/804 [==============================] - 2s 2ms/step - loss: 0.5477 - accura

In [26]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5573 - accuracy: 0.7236 - 1s/epoch - 5ms/step
Loss: 0.5572808980941772, Accuracy: 0.7236151695251465


In [27]:
nn.save('AlphabetSoupCharity_Optimization.h5')